In [1]:
!pip install transformers
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from transformers import DPRContextEncoder, DPRContextEncoderTokenizer
from transformers import DPRQuestionEncoder, DPRQuestionEncoderTokenizer
import torch

# initialize question and context encoders and their tokenizers
question_encoder = DPRQuestionEncoder.from_pretrained('facebook/dpr-question_encoder-single-nq-base')
question_tokenizer = DPRQuestionEncoderTokenizer.from_pretrained('facebook/dpr-question_encoder-single-nq-base')

context_encoder = DPRContextEncoder.from_pretrained('facebook/dpr-ctx_encoder-single-nq-base')
context_tokenizer = DPRContextEncoderTokenizer.from_pretrained('facebook/dpr-ctx_encoder-single-nq-base')

# here is a random context and a related question
context = "Artificial intelligence (AI) is intelligence demonstrated by machines, unlike the natural intelligence displayed by humans and animals, which involves consciousness and emotionality. The distinction between the former and the latter categories is often revealed by the acronym chosen. 'Strong' AI is usually labelled as AGI (Artificial General Intelligence) while attempts to emulate 'natural' intelligence have been called ABI (Artificial Biological Intelligence)."
question = "What is artificial intelligence?"

# tokenize the input (both question and context)
question_inputs = question_tokenizer(question, return_tensors='pt')
context_inputs = context_tokenizer(context, return_tensors='pt')

# encode the input to get embeddings
question_emb = question_encoder(**question_inputs).pooler_output
context_emb = context_encoder(**context_inputs).pooler_output

# compute the similarity between the embeddings (we can use cosine similarity here)
similarity = torch.nn.functional.cosine_similarity(question_emb, context_emb)

print(similarity)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'DPRQuestionEncoderTokenizer'. 
The class this function is called from is 'DPRContextEncoderTokenizer'.


tensor([0.7218], grad_fn=<SumBackward1>)


In [3]:
from datasets import load_dataset
from torch.utils.data import DataLoader
from transformers import DPRQuestionEncoder, DPRQuestionEncoderTokenizerFast
from transformers import DPRContextEncoder, DPRContextEncoderTokenizerFast
from transformers import AdamW
import torch

In [4]:
# Load the 'squad' dataset
squad_dataset = load_dataset('squad')

# Let's consider a subset for demonstration
squad_dataset = {
    'train': squad_dataset['train'].select(range(1000)),
    'validation': squad_dataset['validation'].select(range(500)),
}

  0%|          | 0/2 [00:00<?, ?it/s]

In [5]:
# Load the DPR tokenizers and models
question_tokenizer = DPRQuestionEncoderTokenizerFast.from_pretrained('facebook/dpr-question_encoder-single-nq-base')
ctx_tokenizer = DPRContextEncoderTokenizerFast.from_pretrained('facebook/dpr-ctx_encoder-single-nq-base')

question_model = DPRQuestionEncoder.from_pretrained('facebook/dpr-question_encoder-single-nq-base')
ctx_model = DPRContextEncoder.from_pretrained('facebook/dpr-ctx_encoder-single-nq-base')

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'DPRQuestionEncoderTokenizer'. 
The class this function is called from is 'DPRContextEncoderTokenizerFast'.


In [6]:
def prepare_dataset(example):
    # Tokenize questions and contexts
    question = question_tokenizer(example['question'], truncation=True, padding='max_length', max_length=256)
    context = ctx_tokenizer(example['context'], truncation=True, padding='max_length', max_length=256)

    return {
        'question_input_ids': torch.tensor(question['input_ids']),
        'question_attention_mask': torch.tensor(question['attention_mask']),
        'context_input_ids': torch.tensor(context['input_ids']),
        'context_attention_mask': torch.tensor(context['attention_mask']),
        'labels': torch.tensor(0),  # for SQuAD, we don't have labels as in NQ, so just a dummy label
    }

# Prepare datasets
squad_dataset = {
    'train': squad_dataset['train'].map(prepare_dataset),
    'validation': squad_dataset['validation'].map(prepare_dataset),
}

# Convert to PyTorch DataLoader
dataloader = {
    'train': DataLoader(squad_dataset['train'], batch_size=16, shuffle=True),
    'validation': DataLoader(squad_dataset['validation'], batch_size=16, shuffle=True),
}

In [7]:
# Move models to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
question_model = question_model.to(device)
ctx_model = ctx_model.to(device)

# Initialize optimizer
optimizer = AdamW(list(question_model.parameters()) + list(ctx_model.parameters()), lr=1e-5)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [11]:
# Training loop
for epoch in range(3):
    for i, batch in enumerate(dataloader['train']):
        # Move the batch tensors to the same device as the models
        question_input_ids = torch.stack(batch['question_input_ids']).to(device)
        question_attention_mask = torch.stack(batch['question_attention_mask']).to(device)
        context_input_ids = torch.stack(batch['context_input_ids']).to(device)
        context_attention_mask = torch.stack(batch['context_attention_mask']).to(device)

        optimizer.zero_grad()

        # forward pass
        question_outputs = question_model(input_ids=question_input_ids, attention_mask=question_attention_mask)
        ctx_outputs = ctx_model(input_ids=context_input_ids, attention_mask=context_attention_mask)

        # Calculate the loss
        loss = ((question_outputs.pooler_output - ctx_outputs.pooler_output)**2).mean()

        print(f"Batch {i}, Loss: {loss.item()}")

        # backward pass
        loss.backward()

        optimizer.step()

    print(f"Epoch {epoch} completed.")

Batch 0, Loss: 0.32967501878738403
Batch 1, Loss: 0.17341338098049164
Batch 2, Loss: 0.16407400369644165
Batch 3, Loss: 0.10851892083883286
Batch 4, Loss: 0.09284375607967377
Batch 5, Loss: 0.08512582629919052
Batch 6, Loss: 0.07281264662742615
Batch 7, Loss: 0.07587005943059921
Batch 8, Loss: 0.06397303193807602
Batch 9, Loss: 0.0580436997115612
Batch 10, Loss: 0.05110041797161102
Batch 11, Loss: 0.044356055557727814
Batch 12, Loss: 0.04068971425294876
Batch 13, Loss: 0.03799150139093399
Batch 14, Loss: 0.03221898898482323
Batch 15, Loss: 0.03291810303926468
Batch 16, Loss: 0.02922913245856762
Batch 17, Loss: 0.02888689562678337
Batch 18, Loss: 0.03085346519947052
Batch 19, Loss: 0.02645936608314514
Batch 20, Loss: 0.023925697430968285
Batch 21, Loss: 0.020839959383010864
Batch 22, Loss: 0.024798911064863205
Batch 23, Loss: 0.019117332994937897
Batch 24, Loss: 0.019314497709274292
Batch 25, Loss: 0.018070977181196213
Batch 26, Loss: 0.020015103742480278
Batch 27, Loss: 0.0161894485354